# ✉️ Synthetische E-Mail-Erzeugung für Piranha
Dieses Notebook erzeugt 1.000 (mehr oder weniger) realistisch klingende E-Mails mit PII (random generiert) und generiert passende Trainingsdaten. Die E.Mails werden hier gespeichert /data/piranha_synthetic_data.json"

In [4]:
import random
import json
from faker import Faker
from tqdm import tqdm

faker = Faker("de_DE")
Faker.seed(42)
random.seed(42)

PII_CATEGORIES = ["NAME", "ADRESSE", "VERTRAG", "ZAHLUNG", "TECHNISCHE_DATEN", "KONTAKT", "FIRMA", "DATUM"]

# Beispiel-Templates mit Platzhaltern
TEMPLATES = [
    "Sehr geehrter Herr {NAME}, Ihre Kundennummer {VERTRAG} wird im System geführt.",
    "Die Adresse {ADRESSE} wurde in Ihrem Konto hinterlegt.",
    "Bitte überweisen Sie {ZAHLUNG} bis zum {DATUM}.",
    "Wir haben Ihre Zählernummer {TECHNISCHE_DATEN} registriert.",
    "Rückfragen richten Sie bitte an {KONTAKT}.",
    "Vielen Dank für Ihre Anfrage bei {FIRMA}.",
    "Am {DATUM} fand die letzte Wartung an Zähler {TECHNISCHE_DATEN} statt.",
    "Der Vertrag {VERTRAG} ist seit dem {DATUM} aktiv.",
    "Ihre Telefonnummer {KONTAKT} wurde aktualisiert.",
    "Mit freundlichen Grüßen,\n{NAME}\n{FIRMA}\n{ADRESSE}",
    "Ihr Verbrauch in diesem Monat beträgt {TECHNISCHE_DATEN}.",
    "IBAN {ZAHLUNG} wurde für die Zahlung verwendet.",
    "Ihr Termin mit {NAME} von {FIRMA} ist am {DATUM}.",
    "Sie wohnen aktuell in der {ADRESSE}, korrekt?",
    "Die letzte Abrechnung über {ZAHLUNG} wurde am {DATUM} erstellt.",
]

def generate_pii_values():
    return {
        "NAME": faker.name(),
        "ADRESSE": f"{faker.street_name()} {faker.building_number()}, {faker.postcode()} {faker.city()}",
        "VERTRAG": str(faker.random_number(digits=9, fix_len=True)),
        "ZAHLUNG": f"{round(random.uniform(50, 500), 2)} EUR",
        "TECHNISCHE_DATEN": str(faker.random_number(digits=6, fix_len=True)),
        "KONTAKT": faker.phone_number(),
        "FIRMA": faker.company(),
        "DATUM": faker.date(pattern="%d.%m.%Y")
    }

def generate_synthetic_emails(n=1000):
    dataset = []
    for _ in tqdm(range(n)):
        pii_values = generate_pii_values()
        template = random.choice(TEMPLATES)
        text = template
        labels = []

        for key, value in pii_values.items():
            placeholder = "{" + key + "}"
            if placeholder in template:
                start = text.index(placeholder)
                text = text.replace(placeholder, value, 1)
                labels.append({
                    "start": start,
                    "end": start + len(value),
                    "label": key
                })

        dataset.append({
            "text": text,
            "labels": labels
        })
    return dataset


In [5]:
# ✨ 1.000 E-Mails erzeugen und als JSON speichern
synthetic_dataset = generate_synthetic_emails(n=1000)
output_path = "../data/piranha_synthetic_data.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(synthetic_dataset, f, ensure_ascii=False, indent=2)

print(f"Synthetischer Trainingsdatensatz gespeichert unter: {output_path}")


100%|██████████| 1000/1000 [00:00<00:00, 24694.46it/s]

Synthetischer Trainingsdatensatz gespeichert unter: ../data/piranha_synthetic_data.json
